# Player Search
This notebook explores the use of these two endpoints in retrieving individual NBA player statstics:
- https://v2.nba.api-sports.io/players
- https://v2.nba.api-sports.io/players/statistics

The code from this notebook will be used in V1 of the _Player Search_ feature.

In [1]:
from rel_bball_analytics import app

import pandas as pd
import requests

In [2]:
base_url = app.config["SPORTS_API_URL"]
headers = {"x-rapidapi-key": app.config["SPORTS_API_KEY"]}

### Get Player Info from Name

In [3]:
PLAYER_COLUMNS = [
    "id",
    "firstname",
    "lastname",
    "birth_date",
    "country",
    "height_feet",
    "height_inches",
    "weight_pounds",
    "jersey",
    "is_active",
    "position",
    "start_year",
    "pro_years",
    "college",
]

In [4]:
def get_raw_player_data(name):
    url = f"{base_url}players"
    params = {"name": name}
    
    response = requests.get(url, params=params, headers=headers)
    return response.json()

In [5]:
def filter_nba_players(players):
    players["is_nba"] = players["leagues"].apply(lambda obj: "standard" in obj.keys())
    return players.loc[players.is_nba]

In [6]:
def clean_player_data(data):
    data["birth_date"] = data["birth"].apply(lambda obj: obj.get("date"))
    data["country"] = data["birth"].apply(lambda obj: obj.get("country"))
    data["start_year"] = data["nba"].apply(lambda obj: obj.get("start"))
    data["pro_years"] = data["nba"].apply(lambda obj: obj.get("pro"))
    data["height_feet"] = data["height"].apply(lambda obj: obj.get("feets"))
    data["height_inches"] = data["height"].apply(lambda obj: obj.get("inches"))
    data["weight_pounds"] = data["weight"].apply(lambda obj: obj.get("pounds"))
    data["jersey"] = data["leagues"].apply(lambda obj: obj["standard"].get("jersey"))
    data["is_active"] = data["leagues"].apply(lambda obj: obj["standard"].get("active"))
    data["position"] = data["leagues"].apply(lambda obj: obj["standard"].get("pos"))

    return data[PLAYER_COLUMNS]

In [7]:
data = get_raw_player_data(name="curry")
if data["results"] == 0:
    print("No player matches")

df_results = pd.DataFrame(data["response"])
df_results = filter_nba_players(players=df_results)
df_results = clean_player_data(data=df_results)

In [8]:
df_results

,id,firstname,lastname,birth_date,country,height_feet,height_inches,weight_pounds,jersey,is_active,position,start_year,pro_years,college
0,123,Seth,Curry,1990-08-23,USA,6,2,185,30,True,G,2013,7,Duke
1,124,Stephen,Curry,1988-03-14,USA,6,2,185,30,True,G,2009,12,Davidson


### Get Player Stats from ID

In [9]:
EXCLUDED_STATS_COLUMNS = [
    "player",
    "team",
    "game",
    "comment",
]

In [10]:
def get_raw_stats_data(player_id, season=2022):
    url = f"{base_url}players/statistics"
    params = {"id": player_id, "season": season}
    
    response = requests.get(url, params=params, headers=headers)
    return response.json()

In [11]:
def clean_stats_data(data):
    data["player_id"] = data["player"].apply(lambda obj: obj["id"])
    data["team_id"] = data["team"].apply(lambda obj: obj["id"])
    data["team_name"] = data["team"].apply(lambda obj: obj["name"])
    data["team_code"] = data["team"].apply(lambda obj: obj["code"])
    data["game_id"] = data["game"].apply(lambda obj: obj["id"])

    return data.drop(EXCLUDED_STATS_COLUMNS, axis=1)

In [12]:
data = get_raw_stats_data(player_id=124, season=2019)
if data["results"] == 0:
    print(f"No stats available")

df_results = pd.DataFrame(data["response"])
df_results = df_results.dropna(subset=["points", "min"])
df_results = clean_stats_data(data=df_results)

In [13]:
df_results

,points,pos,min,fgm,fga,fgp,ftm,fta,ftp,tpm,...,pFouls,steals,turnovers,blocks,plusMinus,player_id,team_id,team_name,team_code,game_id
0,18.0,PG,17:57,5.0,11.0,45.5,7.0,8.0,87.5,1.0,...,0.0,2.0,1.0,0.0,-6,124,11,Golden State Warriors,GSW,7625
1,40.0,PG,25:28,14.0,19.0,73.7,6.0,6.0,100,6.0,...,2.0,2.0,5.0,1.0,18,124,11,Golden State Warriors,GSW,7654
2,17.0,PG,21:24,6.0,17.0,35.3,2.0,2.0,100,3.0,...,3.0,2.0,3.0,2.0,-12,124,11,Golden State Warriors,GSW,7677
4,32.0,PG,32:48,9.0,20.0,45.0,8.0,8.0,100,6.0,...,4.0,0.0,2.0,1.0,10,124,11,Golden State Warriors,GSW,7698
5,23.0,PG,29:48,8.0,20.0,40.0,5.0,5.0,100,2.0,...,3.0,1.0,8.0,0.0,-19,124,11,Golden State Warriors,GSW,6405
6,23.0,PG,30:08,7.0,18.0,38.9,7.0,7.0,100,2.0,...,0.0,0.0,2.0,0.0,-30,124,11,Golden State Warriors,GSW,6425
7,26.0,PG,31:00,9.0,17.0,52.9,4.0,4.0,100,4.0,...,5.0,3.0,1.0,0.0,32,124,11,Golden State Warriors,GSW,6436
8,9.0,PG,21:09,3.0,11.0,27.3,2.0,2.0,100,1.0,...,2.0,1.0,4.0,2.0,-13,124,11,Golden State Warriors,GSW,6454
9,0.0,None,0,0.0,0.0,0,0.0,0.0,0,0.0,...,0.0,0.0,0.0,0.0,0,124,11,Golden State Warriors,GSW,6087
12,23.0,PG,27:14,6.0,16.0,37.5,8.0,8.0,100,3.0,...,1.0,0.0,1.0,0.0,-13,124,11,Golden State Warriors,GSW,7318
